# Initialisation

In [12]:
# Importations
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from imblearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from preprocessing import preprocessor as prep
from preprocessing import preprocessor_no_scaler as prep_no_scl
from styles import *

In [2]:
# Initialisation
train = pd.read_csv('../02_data/application_train.csv')
test = pd.read_csv('../02_data/application_test.csv')

id_error_msg = lambda x: '`SK_ID_CURR` is not unic for {} set!'.format(x)
assert len(train.SK_ID_CURR.unique()) == train.shape[0], id_error_msg('train')
assert len(test.SK_ID_CURR.unique()) == test.shape[0], id_error_msg('test')
train.set_index('SK_ID_CURR', inplace=True)
test.set_index('SK_ID_CURR', inplace=True)

print('Training set dimensions :', train.shape)

cls_size = train.TARGET.value_counts()
cls_freq = train.TARGET.value_counts(normalize=True)
print(pd.DataFrame({'size': cls_size,
                    'freq': cls_freq.apply(lambda x: '%.3f' % x)}))

Training set dimensions : (307511, 121)
     size   freq
0  282686  0.919
1   24825  0.081


In [4]:
train_sample = train[::10]
print('Sampled training set dimensions :', train_sample.shape)
cls_size = train_sample.TARGET.value_counts()
cls_freq = train_sample.TARGET.value_counts(normalize=True)
print(pd.DataFrame({'size': cls_size,
                    'freq': cls_freq.apply(lambda x: '%.3f' % x)}))

Sampled training set dimensions : (30752, 121)
    size   freq
0  28303  0.920
1   2449  0.080


On échantillonne le dataset en prenant 10% des points de données

In [5]:
X, y = train.iloc[:, 1:], train.iloc[:, 0]#.values.reshape(-1,1)
Xs, ys = train_sample.iloc[:, 1:], train_sample.iloc[:, 0]#.values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=.2,
                                                    random_state=0)
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (24601, 120)
y_train: (24601,)
X_test: (6151, 120)
y_test: (6151,)


# Rééquilibrage de classes - SMOTE/Tomek

Il y a ~8% de cas de défaut dans le jeu d'entraînement contre 92% de cas sans défaut. Le déséquilibre des classes pose problème dans le cadre de la prédiction de la classe minoritaire par un algorithme de ml.

Il faut rééquilibrer les classes du jeu d'entraînement avant de sélectionner le meilleur modèle de ml 

## Impact de SMOTE Tomek sur la répartition des classes

In [13]:
resamplr = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
udsamplr = SMOTEENN(random_state=42)
rusamplr = RandomUnderSampler(random_state=42)

In [7]:
X_train_trans = prep.fit_transform(X_train)
print(X_train_trans.shape)
print(X_train_trans)
print(y_train.shape)
print(y_train.value_counts())

(24601, 235)
[[0.         0.09011628 0.07823375 ... 1.         0.         0.        ]
 [0.         0.01162791 0.01353611 ... 0.         1.         0.        ]
 [0.         0.05232558 0.15492746 ... 0.         1.         0.        ]
 ...
 [0.         0.14244186 0.1340753  ... 0.         1.         0.        ]
 [0.1        0.12790698 0.28631022 ... 0.         0.         0.        ]
 [0.3        0.06395349 0.25047455 ... 0.         1.         0.        ]]
(24601,)
0    22659
1     1942
Name: TARGET, dtype: int64


In [39]:
X_train_resampl, y_train_resampl = resamplr.fit_resample(X_train_trans, y_train)
print(X_train_resampl.shape)
print(y_train_resampl.value_counts())

(45318, 235)
0    22659
1    22659
Name: TARGET, dtype: int64


In [11]:
X_train_udsampl, y_train_udsampl = udsamplr.fit_resample(X_train_trans, y_train)
print(X_train_udsampl.shape)
print(y_train_udsampl.value_counts())

(33702, 235)
1    22628
0    11074
Name: TARGET, dtype: int64


In [14]:
X_train_rusampl, y_train_rusampl = rusamplr.fit_resample(X_train_trans, y_train)
print(X_train_rusampl.shape)
print(y_train_rusampl.value_counts())

(3884, 235)
0    1942
1    1942
Name: TARGET, dtype: int64


Rééquilibrage exécuté en 1min environ pour un jeu d'entraînement divisé par 10.

## Impact de SMOTE Tomek sur l'entraînement d'un modèle

In [40]:
sgd = Pipeline([('p', prep), ('m', SGDClassifier())])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
scoring = ['precision_macro','recall_macro'] #, 'accuracy']
sgd_scor = cross_validate(sgd, X_train, y_train, scoring=scoring, cv=cv)
print('Model 1\n' + line_decor)
#print('accuracy scores:', sgd_scor['test_accuracy'])
print('precision scores:', sgd_scor['test_precision_macro'])
print('recall scores:', sgd_scor['test_recall_macro'])
#print('Mean Accuracy: %.4f' % np.mean(sgd_scores['test_accuracy']))
print('Mean Precision: %.4f' % np.nanmean(sgd_scor['test_precision_macro']))
print('Mean Recall: %.4f' % np.nanmean(sgd_scor['test_recall_macro']))

Model 1
--------
precision scores: [       nan 0.46056911 0.46056911 0.46056911        nan]
recall scores: [nan 0.5 0.5 0.5 nan]
Mean Precision: 0.4606
Mean Recall: 0.5000


Validation croisée sans SMOTE Tomek : 8.7s avec un échantillon divisé par 10

In [41]:
sgd_imb = Pipeline([('p', prep), ('r', resamplr), ('m', SGDClassifier())])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
scoring = ['precision_macro','recall_macro'] #, 'accuracy']
sgd_imb_scor = cross_validate(sgd_imb, X_train, y_train, scoring=scoring, cv=5)
print('Model 1 - with imbalance handling\n' + line_decor)
#print('accuracy scores:', sgd_imb_scor['test_accuracy'])
print('precision scores:', sgd_imb_scor['test_precision_macro'])
print('recall scores:', sgd_imb_scor['test_recall_macro'])
#print('Mean Accuracy: %.4f' % np.mean(sgd_imb_scores['test_accuracy']))
print('Mean Precision: %.4f' % np.nanmean(sgd_imb_scor['test_precision_macro']))
print('Mean Recall: %.4f' % np.nanmean(sgd_imb_scor['test_recall_macro']))

Model 1 - with imbalance handling
--------
precision scores: [       nan 0.55255999 0.5584412         nan 0.55571135]
recall scores: [       nan 0.66237227 0.63354292        nan 0.67955739]
Mean Precision: 0.5556
Mean Recall: 0.6585


Validation croisée avec SMOTE Tomek (stratégie majoritaire) : 207.6s avec un échantillon divisé par 10

In [7]:
smote_unsmote_ratio = 207.6 / 8.7
print('{:.2f}'.format(smote_unsmote_ratio))

23.86


In [43]:
smote_unsmote_ratio = 186.5 / 9.6
print('{:2f}'.format(smote_unsmote_ratio))

19.427083


Le SMOTE Tomek multiplie par un facteur 19 à 24 le temps d'exécution du modèle

Essai d'une validation croisée sans SMOTE Tomek avec tous les points du jeu d'entraînement

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (246008, 120)
y_train: (246008, 1)
X_test: (61503, 120)
y_test: (61503, 1)


In [9]:
sgd = Pipeline([('p', prep), ('m', SGDClassifier())])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
scoring = ['precision_macro','recall_macro'] #, 'accuracy']
sgd_scor = cross_validate(sgd, X_train, y_train, scoring=scoring, cv=cv)
print('Model 1\n' + line_decor)
#print('accuracy scores:', sgd_scor['test_accuracy'])
print('precision scores:', sgd_scor['test_precision_macro'])
print('recall scores:', sgd_scor['test_recall_macro'])
#print('Mean Accuracy: %.4f' % np.mean(sgd_scores['test_accuracy']))
print('Mean Precision: %.4f' % np.nanmean(sgd_scor['test_precision_macro']))
print('Mean Recall: %.4f' % np.nanmean(sgd_scor['test_recall_macro']))

Model 1
--------
precision scores: [0.45967644 0.45966627 0.45966627 0.45967562 0.45967562]
recall scores: [0.5 0.5 0.5 0.5 0.5]
Mean Precision: 0.4597
Mean Recall: 0.5000


Validation croisée sans SMOTE Tomek exécutée en 57.9s sur tout le jeu de données

In [11]:
unsampled_sampled_ratio = 57.9 / 8.7
print('{:.2f}'.format(unsampled_sampled_ratio))

6.66


Il faut 7 fois plus de temps pour exécuter la même chose sur 10 fois plus de données (pas parfaitement linéaire donc)

In [12]:
print('{:.2f}'.format(207.6 * unsampled_sampled_ratio))

1381.61


In [13]:
1381 / 60

23.016666666666666

Il faudrait 23 minutes rien que pour faire du rééquilibrage avec le jeu de données actuel. Pas souhaitable. 

**Il faut trouver un moyen de raccourcir le temps d'exécution du rééquilibrage.**

## Réduction du temps de rééquilibrage en suppprimant des colonnes

In [45]:
X_train_resampl_cut, y_train_resampl_cut = resamplr.fit_resample(
    X_train_trans[:,:50], y_train
    )
print(X_train_resampl_cut.shape)
print(y_train_resampl_cut.value_counts())

(45313, 50)
1    22659
0    22654
Name: TARGET, dtype: int64


temps d'entraînement 52s pour un jeu d'entraînement divisé par 10 avec seulement les 50 premières colonnes contre 60.5s avec toutes les colonnes.

# Sous-échantillonage aléatoire

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X_train: (246008, 120)
y_train: (246008,)
X_test: (61503, 120)
y_test: (61503,)


In [16]:
sgd_imb = Pipeline([('p', prep), ('r', rusamplr), ('m', SGDClassifier())])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
scoring = ['precision_macro','recall_macro'] #, 'accuracy']
sgd_imb_scor = cross_validate(sgd_imb, X_train, y_train, scoring=scoring, cv=5)
print('Model 1 - with imbalance handling\n' + line_decor)
#print('accuracy scores:', sgd_imb_scor['test_accuracy'])
print('precision scores:', sgd_imb_scor['test_precision_macro'])
print('recall scores:', sgd_imb_scor['test_recall_macro'])
#print('Mean Accuracy: %.4f' % np.mean(sgd_imb_scores['test_accuracy']))
print('Mean Precision: %.4f' % np.nanmean(sgd_imb_scor['test_precision_macro']))
print('Mean Recall: %.4f' % np.nanmean(sgd_imb_scor['test_recall_macro']))

Model 1 - with imbalance handling
--------
precision scores: [0.54163367        nan 0.56050468 0.55293874        nan]
recall scores: [0.62721639        nan 0.67366715 0.67118886        nan]
Mean Precision: 0.5517
Mean Recall: 0.6574


# Modèle 1 : SGD Classifier

In [3]:
model1 = Pipeline([('p', prep), ('m', SGDClassifier())])
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print('Model 1\n' + line_decor)
print('Score: %.4f' % model1.score(X_test, y_test))
print(line_decor + '\nConfusion matrix\n' + str(conf_mat))
print(classification_report(y_test, y_pred))

Model 1
--------
Score: 0.9190
--------
Confusion matrix
[[56522     0]
 [ 4981     0]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56522
           1       0.00      0.00      0.00      4981

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.84      0.92      0.88     61503



# Modèle 2 : Random Forest Classifier

In [13]:
model2 = Pipeline([('p', prep_no_scl), ('m', RandomForestClassifier())])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scoring = ['accuracy','precision_macro','recall_macro']
scores_model2 = cross_validate(model2, X_train, y_train, scoring=scoring, cv=cv,
                               n_jobs=-1)

print('Model 2\n' + 8 * '-')
print('Mean Accuracy: %.4f' % np.mean(scores_model2['test_accuracy']))
print('Mean Precision: %.4f' % np.mean(scores_model2['test_precision_macro']))
print('Mean Recall: %.4f' % np.mean(scores_model2['test_recall_macro']))

In [10]:
model2 = Pipeline([('p', prep_no_scl), ('m', RandomForestClassifier())])
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print('Model 2\n' + 8 * '-')
print('Score: %.4f' % model2.score(X_test, y_test))
print(8 * '-' + '\nConfusion matrix\n' + str(conf_mat))
print(classification_report(y_test, y_pred))

Model 1
--------
Score: 0.9185
--------
Confusion matrix
[[56485     4]
 [ 5011     3]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56489
           1       0.43      0.00      0.00      5014

    accuracy                           0.92     61503
   macro avg       0.67      0.50      0.48     61503
weighted avg       0.88      0.92      0.88     61503



In [ ]:
# undersmpling 
# foret d'arbre -> feature importance
# lightgbm
# si besoin pca ou autre

# optimisation du threshold
# flask

In [9]:
y_pred = model2.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[56512     5]
 [ 4979     7]]


In [ ]:
model2.get_params()

# Modèle 3 : LightGBM

In [7]:
model3 = Pipeline([('p', prep), ('m', LGBMClassifier())])
model3.fit(X_train, y_train)
print('Score:', model3.score(X_test, y_test))

Score: 0.9192071931450498


In [8]:
y_pred = model3.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[56447    81]
 [ 4888    87]]


In [9]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56528
           1       0.52      0.02      0.03      4975

    accuracy                           0.92     61503
   macro avg       0.72      0.51      0.50     61503
weighted avg       0.89      0.92      0.88     61503



In [ ]:
# à faire

# smote tomek
# random search precision des deux classes (privilégier light_gbm)
# 
# choisir optimisation recall(classe 1)
# fonction coût : manque à gagner pour chaque treshold
# treshold = + = + precision - recall
# precision élevée = on accepte tout le monde
# recall élevée = on refuse tout le monde
# regarder crer une colonne intérêts (amt credit - good price),
# optimiser mon threshold % de ça

# 2021-09-30 : Modélisation avec sous-échantillonage aléatoire de la classe majoriaire

In [12]:
# Importations
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from preprocessing import preprocessor as prep
from preprocessing import preprocessor_no_scaler as prep_no_scl
from preprocessing import CreditInfosImputer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Initialisation
train = pd.read_csv('../02_data/application_train.csv')
#test = pd.read_csv('../02_data/application_test.csv')

id_error_msg = lambda x: '`SK_ID_CURR` is not unic for {} set!'.format(x)
assert len(train.SK_ID_CURR.unique()) == train.shape[0], id_error_msg('train')
#assert len(test.SK_ID_CURR.unique()) == test.shape[0], id_error_msg('test')
train.set_index('SK_ID_CURR', inplace=True)
#test.set_index('SK_ID_CURR', inplace=True)

print('Training set dimensions :', train.shape)
df = train.copy()

cls_size = df.TARGET.value_counts()
cls_freq = df.TARGET.value_counts(normalize=True)
print(pd.DataFrame({'size': cls_size,
                    'freq': cls_freq.apply(lambda x: '%.3f' % x)}))

Training set dimensions : (307511, 121)
     size   freq
0  282686  0.919
1   24825  0.081


## Test de CreditInfosImputer

### Tout seul

In [3]:
credit_imputer = CreditInfosImputer()

credit_imputer.fit(df)

CreditInfosImputer()

In [6]:
df = train.copy()
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0],
                                                    test_size=.2)

In [9]:
credit_imputer.fit_transform(X_train, y_train)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
346746,Cash loans,F,N,Y,0,103500.0,78192.0,6399.0,67500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
123400,Cash loans,F,N,Y,0,85500.0,314100.0,13833.0,225000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
371653,Cash loans,F,N,Y,0,247500.0,1059781.5,56592.0,954000.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0
324835,Cash loans,M,Y,Y,0,427500.0,675000.0,49117.5,675000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
429236,Revolving loans,M,Y,Y,1,135000.0,270000.0,13500.0,270000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447394,Cash loans,F,N,N,0,81000.0,135000.0,10665.0,135000.0,Family,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
210991,Cash loans,M,N,N,0,112500.0,76500.0,5670.0,76500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
112635,Cash loans,M,Y,Y,0,157500.0,454500.0,23206.5,454500.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
credit_imputer.fit_transform(df)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0


### Dans une pipeline de prétraitements

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0],
                                                    test_size=.2)

train_prep = prep.fit_transform(X_train, y_train)
print(train_prep.shape)
#print(train_prep.shape)

(246008, 237)


In [4]:
train_prep[:5]

array([[0.07041798, 0.06742717, 0.05723906, ..., 0.        , 0.        ,
        0.        ],
       [0.15842697, 0.2458231 , 0.15937149, ..., 0.        , 0.        ,
        0.        ],
       [0.04719101, 0.03624079, 0.04826038, ..., 0.        , 1.        ,
        0.        ],
       [0.12282584, 0.09124254, 0.10549944, ..., 0.        , 1.        ,
        0.        ],
       [0.02247191, 0.04956125, 0.02356902, ..., 0.        , 0.        ,
        0.        ]])

In [5]:
from preprocessing import get_preprocessed_set_column_names as get_feat_names

print(get_feat_names(prep))

['AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXP

## Test de Random Undersampler

In [3]:
rand_usampl = RandomUnderSampler()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0],
                                                    test_size=.2)
resampling = rand_usampl.fit_resample(X_train, y_train)

In [9]:
resampling[0].shape

(39798, 120)

In [11]:
resampling[1].value_counts()

0    19899
1    19899
Name: TARGET, dtype: int64

## Essais avec un classifieur en arbre de décision

In [4]:
tree_imb = Pipeline(steps=[
    ('r', rand_usampl),
    ('p', prep_no_scl),
    ('m', DecisionTreeClassifier())
    ])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0],
                                                    test_size=.2)

In [6]:
tree_imb.fit(X_train, y_train)

Pipeline(steps=[('r', RandomUnderSampler()),
                ('p',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('creditinfosimputer',
                                                  CreditInfosImputer(),
                                                  ['AMT_CREDIT', 'AMT_ANNUITY',
                                                   'AMT_GOODS_PRICE']),
                                                 ('simpleimputer-1',
                                                  SimpleImputer(strategy='median'),
                                                  ['CNT_CHILDREN',
                                                   'AMT_INCOME_TOTAL',
                                                   'REGION_POPULATION_RELATIVE',
                                                   'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGI...
       

In [7]:
y_pred = tree_imb.predict(X_test)

In [8]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.59      0.72     56559
           1       0.11      0.60      0.19      4944

    accuracy                           0.59     61503
   macro avg       0.53      0.59      0.46     61503
weighted avg       0.88      0.59      0.68     61503



In [13]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[33287 23272]
 [ 1997  2947]]


# 2021-10-01 : Selection du meilleur modèle

In [8]:
# Importations
import sys
sys.path.append('..')

# Bibliothèques utiles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

# Prétraitements et rééquilibrage
from preprocessing import preprocessor, preprocessor_no_scaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Modèles
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier

# Évaluation
from sklearn.metrics import classification_report, confusion_matrix

# Autres
from timer import timer
from styles import *

In [9]:
# Initialisation
train = pd.read_csv('../02_data/application_train.csv', index_col=0)
#test = pd.read_csv('../02_data/application_test.csv')

print('Training set dimensions :', train.shape)
df = train.copy()

cls_size = df.TARGET.value_counts()
cls_freq = df.TARGET.value_counts(normalize=True)
print(pd.DataFrame({'size': cls_size,
                    'freq': cls_freq.apply(lambda x: '%.2f' % x)}))

Training set dimensions : (307511, 121)
     size  freq
0  282686  0.92
1   24825  0.08


In [10]:
df.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Définition des modèles à tester

# Pour les besoin de l'évaluation, on fige l'aléatoire
# On définit un nombre pour la graine d'aléa
r = 42

undersampler = RandomUnderSampler(random_state=r)

decision_tree = Pipeline([('u', undersampler),
                          ('p', preprocessor_no_scaler),
                          ('m', DecisionTreeClassifier(random_state=r))])

random_forest = Pipeline([('u', undersampler),
                          ('p', preprocessor_no_scaler),
                          ('m', RandomForestClassifier(random_state=r))])

ada_boost = Pipeline([('u', undersampler),
                      ('p', preprocessor_no_scaler),
                      ('m', AdaBoostClassifier(random_state=r))])

light_gbm = Pipeline([('u', undersampler),
                      ('p', preprocessor_no_scaler),
                      ('m', LGBMClassifier(random_state=r))])

# Liste des modèles à tester
models = {'decision_tree': decision_tree,
          'random_forest': random_forest,
          'ada_boost': ada_boost,
          'light_gbm': light_gbm}


In [13]:
# Séparation du jeu de données entre entraînement et évaluation

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0],
                                                    test_size=.2,
                                                    random_state=r)

In [14]:
# Fonction d'évaluation des modèles
@timer
def model_eval(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [15]:
# Boucle d'évaluation des modèles
for model_name, model in models.items():
    print(model_name)
    model_eval(model.fit(X_train, y_train), X_test, y_test)

decision_tree
[[33054 23500]
 [ 1988  2961]]
              precision    recall  f1-score   support

           0       0.94      0.58      0.72     56554
           1       0.11      0.60      0.19      4949

    accuracy                           0.59     61503
   macro avg       0.53      0.59      0.46     61503
weighted avg       0.88      0.59      0.68     61503

'model_eval': successfully processed in 0h00m01.463322s.
random_forest
[[39363 17191]
 [ 1668  3281]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     56554
           1       0.16      0.66      0.26      4949

    accuracy                           0.69     61503
   macro avg       0.56      0.68      0.53     61503
weighted avg       0.90      0.69      0.76     61503

'model_eval': successfully processed in 0h00m04.185277s.
ada_boost
[[38574 17980]
 [ 1612  3337]]
              precision    recall  f1-score   support

           0       0.96      0.68      0.80   

## Sélection des meilleures variables

In [7]:
from preprocessing import get_preprocessed_set_column_names as get_feat_names

def get_feature_importances(model):
    '''Fonction qui retourne l'importance relative des variables
    pour un modèle donné et un jeu d'entraînement donné'''
    feat_names = get_feat_names(model['p'])
    feat_impor = model['m'].feature_importances_

    importances = pd.Series(data={k:v for k,v in zip(feat_names,feat_impor)},
                            index=feat_names)
    
    return importances

In [9]:
models_feat_importances = []
for model in models.values():
    feat_importances = get_feature_importances(model.fit(X_train, y_train))
    models_feat_importances.append(feat_importances)

In [25]:
feature_importances = pd.DataFrame(np.asarray(models_feat_importances).T,
                                   columns=models.keys(),
                                   index=feat_importances.index)

In [26]:
feature_importances

,decision_tree,random_forest,ada_boost,light_gbm
AMT_CREDIT,0.028085,0.030102,0.04,164.0
AMT_ANNUITY,0.037893,0.029764,0.06,126.0
AMT_GOODS_PRICE,0.020147,0.027110,0.04,125.0
CNT_CHILDREN,0.003156,0.006277,0.00,4.0
AMT_INCOME_TOTAL,0.026114,0.022582,0.00,59.0
...,...,...,...,...
WALLSMATERIAL_MODE_others,0.000268,0.000255,0.00,3.0
WALLSMATERIAL_MODE_panel,0.001925,0.001424,0.00,3.0
WALLSMATERIAL_MODE_stone_or_brick,0.000730,0.001638,0.00,5.0
WALLSMATERIAL_MODE_unknown,0.000796,0.001221,0.00,0.0


In [28]:
feature_importances.random_forest.sort_values(ascending=False)[:10]

EXT_SOURCE_3              0.066009
EXT_SOURCE_2              0.058289
DAYS_BIRTH                0.035045
DAYS_ID_PUBLISH           0.031081
DAYS_EMPLOYED             0.031006
DAYS_REGISTRATION         0.030382
AMT_CREDIT                0.030102
AMT_ANNUITY               0.029764
DAYS_LAST_PHONE_CHANGE    0.029719
EXT_SOURCE_1              0.029667
Name: random_forest, dtype: float64

In [75]:
feature_importances.loc[feature_importances.random_forest>.01, 'random_forest']

AMT_CREDIT                    0.030102
AMT_ANNUITY                   0.029764
AMT_GOODS_PRICE               0.027110
AMT_INCOME_TOTAL              0.022582
REGION_POPULATION_RELATIVE    0.024402
DAYS_BIRTH                    0.035045
DAYS_EMPLOYED                 0.031006
DAYS_REGISTRATION             0.030382
DAYS_ID_PUBLISH               0.031081
OWN_CAR_AGE                   0.013606
HOUR_APPR_PROCESS_START       0.020213
EXT_SOURCE_1                  0.029667
EXT_SOURCE_2                  0.058289
EXT_SOURCE_3                  0.066009
OBS_30_CNT_SOCIAL_CIRCLE      0.011771
OBS_60_CNT_SOCIAL_CIRCLE      0.011660
DAYS_LAST_PHONE_CHANGE        0.029719
AMT_REQ_CREDIT_BUREAU_YEAR    0.014977
WEEKDAY_APPR_PROCESS_START    0.015063
Name: random_forest, dtype: float64

In [31]:
feature_importances.random_forest.sort_values(ascending=True)[:20]

FLAG_DOCUMENT_12                       0.000000e+00
FLAG_MOBIL                             0.000000e+00
FLAG_DOCUMENT_10                       0.000000e+00
NAME_INCOME_TYPE_student               7.755177e-07
NAME_INCOME_TYPE_maternity_leave       8.323829e-07
FLAG_DOCUMENT_4                        8.579669e-07
ORGANIZATION_TYPE_trade_type_5         9.448861e-06
ORGANIZATION_TYPE_industry_type_8      1.011750e-05
FLAG_DOCUMENT_17                       1.314697e-05
FLAG_DOCUMENT_2                        1.463591e-05
NAME_EDUCATION_TYPE_academic_degree    1.693411e-05
NAME_INCOME_TYPE_unemployed            1.808568e-05
FLAG_DOCUMENT_7                        1.831941e-05
ORGANIZATION_TYPE_trade_type_4         2.260609e-05
ORGANIZATION_TYPE_religion             2.468980e-05
FLAG_DOCUMENT_21                       2.786900e-05
ORGANIZATION_TYPE_industry_type_13     2.858213e-05
FLAG_DOCUMENT_20                       2.930056e-05
FLAG_DOCUMENT_19                       3.994547e-05
ORGANIZATION

In [39]:
feature_importances.filter(like='FLAG', axis=0)

,decision_tree,random_forest,ada_boost,light_gbm
FLAG_OWN_CAR,0.002822,4.525899e-03,0.00,20.0
FLAG_OWN_REALTY,0.003491,4.594735e-03,0.00,4.0
FLAG_MOBIL,0.000000,0.000000e+00,0.00,0.0
FLAG_EMP_PHONE,0.000809,1.878337e-03,0.00,0.0
FLAG_WORK_PHONE,0.002898,3.960218e-03,0.00,13.0
FLAG_CONT_MOBILE,0.000000,1.172968e-04,0.00,0.0
FLAG_PHONE,0.003093,4.482902e-03,0.00,9.0
FLAG_EMAIL,0.002243,1.935600e-03,0.00,5.0
FLAG_DOCUMENT_2,0.000084,1.463591e-05,0.00,0.0
FLAG_DOCUMENT_3,0.002904,4.808394e-03,0.02,20.0


In [64]:
feat_importances[[f for f in feat_importances.index
                  if f[-4:] in ['_AVG','MEDI','MODE']]]

APARTMENTS_AVG                  0.007541
BASEMENTAREA_AVG                0.006809
YEARS_BEGINEXPLUATATION_AVG     0.008415
YEARS_BUILD_AVG                 0.005340
COMMONAREA_AVG                  0.005424
ELEVATORS_AVG                   0.003278
ENTRANCES_AVG                   0.005277
FLOORSMAX_AVG                   0.004091
FLOORSMIN_AVG                   0.003219
LANDAREA_AVG                    0.007067
LIVINGAPARTMENTS_AVG            0.004940
LIVINGAREA_AVG                  0.008482
NONLIVINGAPARTMENTS_AVG         0.002860
NONLIVINGAREA_AVG               0.005850
APARTMENTS_MEDI                 0.007673
BASEMENTAREA_MEDI               0.006966
YEARS_BEGINEXPLUATATION_MEDI    0.008135
YEARS_BUILD_MEDI                0.005243
COMMONAREA_MEDI                 0.005467
ELEVATORS_MEDI                  0.002044
ENTRANCES_MEDI                  0.004274
FLOORSMAX_MEDI                  0.003503
FLOORSMIN_MEDI                  0.002558
LANDAREA_MEDI                   0.007450
LIVINGAPARTMENTS

* Les variables type `FLAG` semblent peu impactantes pour des modèles en arbre de décision
* Il faut réduire les nombres de colonnes onehot pour les types de métiers et
d'organisation car la cardinalité de ces variables est trop grande. Certaines catégories d'organisations ou de métiers sont trop spécifiques et peuvent être regroupées

In [44]:
df.ORGANIZATION_TYPE.unique()

array(['Business Entity Type 3', 'School', 'Government', 'Religion',
       'Other', 'XNA', 'Electricity', 'Medicine',
       'Business Entity Type 2', 'Self-employed', 'Transport: type 2',
       'Construction', 'Housing', 'Kindergarten', 'Trade: type 7',
       'Industry: type 11', 'Military', 'Services', 'Security Ministries',
       'Transport: type 4', 'Industry: type 1', 'Emergency', 'Security',
       'Trade: type 2', 'University', 'Transport: type 3', 'Police',
       'Business Entity Type 1', 'Postal', 'Industry: type 4',
       'Agriculture', 'Restaurant', 'Culture', 'Hotel',
       'Industry: type 7', 'Trade: type 3', 'Industry: type 3', 'Bank',
       'Industry: type 9', 'Insurance', 'Trade: type 6',
       'Industry: type 2', 'Transport: type 1', 'Industry: type 12',
       'Mobile', 'Trade: type 1', 'Industry: type 5', 'Industry: type 10',
       'Legal Services', 'Advertising', 'Trade: type 5', 'Cleaning',
       'Industry: type 13', 'Trade: type 4', 'Telecom',
       'I

In [62]:
df.loc[df.ORGANIZATION_TYPE.str.match(r'^Industry'),
       'ORGANIZATION_TYPE'] = 'Industry'
df.loc[df.ORGANIZATION_TYPE.str.match(r'^Transport'),
        'ORGANIZATION_TYPE'] = 'Transport'
df.loc[df.ORGANIZATION_TYPE.str.match(r'^Trade'),
        'ORGANIZATION_TYPE'] = 'Trade'
df.loc[df.ORGANIZATION_TYPE.str.match(r'^Business Entity'),
        'ORGANIZATION_TYPE'] = 'Business Entity'

In [64]:
print(df.ORGANIZATION_TYPE.unique())
print(len(df.ORGANIZATION_TYPE.unique()))

['Business Entity' 'School' 'Government' 'Religion' 'Other' 'XNA'
 'Electricity' 'Medicine' 'Self-employed' 'Transport' 'Construction'
 'Housing' 'Kindergarten' 'Trade' 'Industry' 'Military' 'Services'
 'Security Ministries' 'Emergency' 'Security' 'University' 'Police'
 'Postal' 'Agriculture' 'Restaurant' 'Culture' 'Hotel' 'Bank' 'Insurance'
 'Mobile' 'Legal Services' 'Advertising' 'Cleaning' 'Telecom' 'Realtor']
35


In [65]:
df.ORGANIZATION_TYPE.value_counts()

Business Entity        84529
XNA                    55374
Self-employed          38412
Other                  16683
Trade                  14315
Industry               14311
Medicine               11193
Government             10404
Transport               8990
School                  8893
Kindergarten            6880
Construction            6721
Security                3247
Housing                 2958
Military                2634
Bank                    2507
Agriculture             2454
Police                  2341
Postal                  2157
Security Ministries     1974
Restaurant              1811
Services                1575
University              1327
Hotel                    966
Electricity              950
Insurance                597
Telecom                  577
Emergency                560
Advertising              429
Realtor                  396
Culture                  379
Mobile                   317
Legal Services           305
Cleaning                 260
Religion      

In [66]:
preprocessor_no_scaler.fit_transform(df).shape

(307511, 215)

In [68]:
# Séparation du jeu de données entre entraînement et évaluation

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0],
                                                    test_size=.2,
                                                    random_state=r)

In [69]:
# Boucle d'évaluation des modèles
for model_name, model in models.items():
    print(model_name)
    model_eval(model.fit(X_train, y_train), X_test, y_test)

decision_tree
[[33134 23420]
 [ 2030  2919]]
              precision    recall  f1-score   support

           0       0.94      0.59      0.72     56554
           1       0.11      0.59      0.19      4949

    accuracy                           0.59     61503
   macro avg       0.53      0.59      0.45     61503
weighted avg       0.88      0.59      0.68     61503

'model_eval': successfully processed in 0h00m01.840444s.
random_forest
[[39346 17208]
 [ 1713  3236]]
              precision    recall  f1-score   support

           0       0.96      0.70      0.81     56554
           1       0.16      0.65      0.25      4949

    accuracy                           0.69     61503
   macro avg       0.56      0.67      0.53     61503
weighted avg       0.89      0.69      0.76     61503

'model_eval': successfully processed in 0h00m03.689332s.
ada_boost
[[38542 18012]
 [ 1614  3335]]
              precision    recall  f1-score   support

           0       0.96      0.68      0.80   

In [70]:
new_importances = get_feature_importances(random_forest)

In [72]:
new_importances.sort_values(ascending=False)[:20]

EXT_SOURCE_3                  0.066164
EXT_SOURCE_2                  0.061207
DAYS_BIRTH                    0.034511
DAYS_EMPLOYED                 0.031532
DAYS_ID_PUBLISH               0.031449
DAYS_LAST_PHONE_CHANGE        0.030263
AMT_ANNUITY                   0.030164
EXT_SOURCE_1                  0.030088
DAYS_REGISTRATION             0.030017
AMT_CREDIT                    0.029999
AMT_GOODS_PRICE               0.027940
REGION_POPULATION_RELATIVE    0.023971
AMT_INCOME_TOTAL              0.023164
HOUR_APPR_PROCESS_START       0.020020
WEEKDAY_APPR_PROCESS_START    0.015229
AMT_REQ_CREDIT_BUREAU_YEAR    0.014610
OWN_CAR_AGE                   0.013775
OBS_30_CNT_SOCIAL_CIRCLE      0.011943
OBS_60_CNT_SOCIAL_CIRCLE      0.011638
TOTALAREA_MODE                0.009789
dtype: float64

In [73]:
new_importances.sort_values(ascending=True)[:20]

FLAG_MOBIL                             0.000000
FLAG_DOCUMENT_12                       0.000000
FLAG_DOCUMENT_4                        0.000000
FLAG_DOCUMENT_10                       0.000000
NAME_INCOME_TYPE_maternity_leave       0.000002
NAME_INCOME_TYPE_student               0.000003
FLAG_DOCUMENT_7                        0.000007
FLAG_DOCUMENT_17                       0.000011
NAME_EDUCATION_TYPE_academic_degree    0.000013
NAME_INCOME_TYPE_unemployed            0.000015
FLAG_DOCUMENT_2                        0.000019
FLAG_DOCUMENT_21                       0.000021
FLAG_DOCUMENT_20                       0.000026
ORGANIZATION_TYPE_religion             0.000027
FLAG_DOCUMENT_19                       0.000042
ORGANIZATION_TYPE_cleaning             0.000053
OCCUPATION_TYPE_hr_staff               0.000063
FLAG_DOCUMENT_15                       0.000069
ORGANIZATION_TYPE_legal_services       0.000072
ORGANIZATION_TYPE_realtor              0.000075
dtype: float64

## Optimisation des hypers-paramètres

On va optimiser les hypers-paramètres de light-gbm avec une RandomizedSearchCV

In [ ]:
# hyper optimisation (randomized search / hyperopt?)
# (lien vers hyperopt : https://www.kaggle.com/shishu1421/lightgbm-using-hyperopt)
# calcul seuil de décision basé sur coût crédit
# Commencer la partie Flask
# Streamlit (pour la partie dashboard web [https://streamlit.io/])